In [1]:
%matplotlib inline
import pandas as pd
import itertools
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import bnpy
import warnings
from scipy import stats
import random

from scipy.stats import kruskal, f_oneway
import collections
import scikit_posthocs as sp

warnings.filterwarnings('ignore')

In [2]:
pth = '../data/expression/target-high-risk-nbl-mycn-na-exp-2018-11-12.tsv'

exp = pd.read_csv(pth, sep='\t', index_col=0) 

In [3]:
pth = '../data/archive/mycn-na-nbl-0.2-assign-2019-03-02.tsv'

assign = pd.read_csv(pth, sep='\t', index_col=0)

In [4]:
exp.reindex(assign[assign['1'] == 1].index.values, axis=1).to_csv('target-high-risk-nbl-mycn-na-exp-2018-11-12-cluster0.tsv',
                                                                  sep='\t')

In [5]:
def n1(key, zscore):
    import uuid
    import subprocess
    import numpy as np
    
    _dir = '/tmp/%s' % str(uuid.uuid4())

    os.mkdir(_dir)

    cmd = ["Rscript",
           "/opt/hydra/bin/fgsea.R",
           "/opt/hydra/gene-sets/Human_GOBP_AllPathways_no_GO_iea_December_01_2018_symbol.gmt",
           os.path.join(_dir, "%s.rnk" % key),
           os.path.join(_dir, "%s.fgsea" % key)]

    zscore = zscore.sort_values(ascending=False)
    
    zscore.to_csv(os.path.join(_dir, "%s.rnk" % key),
                 header=None,
                 sep='\t')

    subprocess.check_call(cmd)
    
    return pd.read_csv(os.path.join(_dir, "%s.fgsea" % key), index_col=0)

In [6]:
RUN = False

if RUN: 
    _dir = '../data/gsea/'
    for cluster, rows in assign.groupby('1'):
        samps = rows.index.values
        back = [x for x in exp.columns if x not in samps]
    
        for samp in samps:
            zscore = (exp[samp] - exp[back].mean(axis=1)) / (exp[back].std(axis=1) + 0.05)
            res = n1(samp, zscore)
        
            pth = os.path.join(_dir, 'sub_cohort', str(cluster), samp)
            res.to_csv(pth)
        
            cback = [x for x in samps if x != samp]
            zscore = (exp[samp] - exp[cback].mean(axis=1)) / (exp[cback].std(axis=1) + 0.05)
            res2 = n1(samp, zscore)
        
            pth = os.path.join(_dir, 'sub_clust', str(cluster), samp)
            res2.to_csv(pth)
            
else:
    print("Run already")

Run already


In [7]:
_dir = '../data/gsea/'

sub_cohort = collections.defaultdict(list)
sub_cohort_unique = collections.defaultdict(list)
sub_clust_unique = collections.defaultdict(list)

sigs = set()

interest = ["REGULATION OF VASCULATURE DEVELOPMENT%GOBP%GO:1901342", 
            "POSITIVE REGULATION OF LOCOMOTION%GOBP%GO:0040017",
            "PLATELET DEGRANULATION%GOBP%GO:0002576",
            "RESPONSE TO ELEVATED PLATELET CYTOSOLIC CA2+%REACTOME DATABASE ID RELEASE 66%76005",
            "POSITIVE REGULATION OF VASCULATURE DEVELOPMENT%GOBP%GO:1904018"]

for cluster, rows in assign.groupby('1'):
    samps = rows.index.values
    for samp in samps:
        pth = os.path.join(_dir, 'sub_cohort', str(cluster), samp)        
        f1 = pd.read_csv(pth, index_col=0)
        f1 = f1[(f1['padj'] < 0.05) & (f1['ES'] > 0.0)]
        sigs.update(f1.index.values)
        
        pth = os.path.join(_dir, 'sub_clust', str(cluster), samp)
        f2 = pd.read_csv(pth, index_col=0)
        f2 = f2[(f2['padj'] < 0.05) & (f2['ES'] > 0.0)]
        sigs.update(f2.index.values)
        
        if cluster == 1:
            print samp
            print f1.sort_values('NES', ascending=False)['NES'].head(50)
            print '\n'
            print f2.sort_values('NES', ascending=False)['NES'].head(50)
            print '\n\n'
            
            if len(f2.index.intersection(interest)) > 0:
                cluster1_metastatic.append(samp)
        
        sub_cohort[cluster].extend(list(f1.index.values))
        sub_cohort_unique[cluster].extend(list(set(f1.index.values) - set(f2.index.values)))
        sub_clust_unique[cluster].extend(list(set(f2.index.values) - set(f1.index.values)))

TARGET-30-PALETP-01
pathway
COMPLEMENT ACTIVATION, CLASSICAL PATHWAY%GOBP%GO:0006958                                                                     4.787501
COMPLEMENT ACTIVATION%GOBP%GO:0006956                                                                                        4.755103
HUMORAL IMMUNE RESPONSE MEDIATED BY CIRCULATING IMMUNOGLOBULIN%GOBP%GO:0002455                                               4.746461
B CELL RECEPTOR SIGNALING PATHWAY%GOBP%GO:0050853                                                                            4.611476
PHAGOCYTOSIS, RECOGNITION%GOBP%GO:0006910                                                                                    4.383621
IMMUNOGLOBULIN MEDIATED IMMUNE RESPONSE%GOBP%GO:0016064                                                                      4.382914
B CELL MEDIATED IMMUNITY%GOBP%GO:0019724                                                                                     4.359972
PROTEIN ACTIVATION CASCADE%GOBP%GO

TARGET-30-PALTEG-01
pathway
EUKARYOTIC TRANSLATION ELONGATION%REACTOME DATABASE ID RELEASE 66%156842                                                                        7.153070
PEPTIDE CHAIN ELONGATION%REACTOME DATABASE ID RELEASE 66%156902                                                                                 7.103036
FORMATION OF A POOL OF FREE 40S SUBUNITS%REACTOME DATABASE ID RELEASE 66%72689                                                                  7.029831
VIRAL MRNA TRANSLATION%REACTOME%R-HSA-192823.3                                                                                                  6.943775
EUKARYOTIC TRANSLATION TERMINATION%REACTOME%R-HSA-72764.4                                                                                       6.875884
CYTOPLASMIC RIBOSOMAL PROTEINS%WIKIPATHWAYS_20181110%WP477%HOMO SAPIENS                                                                         6.803590
SELENOCYSTEINE SYNTHESIS%REACTOME%R-HSA-2408557.2     

TARGET-30-PANNMS-01
pathway
HALLMARK_E2F_TARGETS%MSIGDB_C2%HALLMARK_E2F_TARGETS                                                                         4.096941
HALLMARK_MYC_TARGETS_V1%MSIGDB_C2%HALLMARK_MYC_TARGETS_V1                                                                   3.596211
HALLMARK_G2M_CHECKPOINT%MSIGDB_C2%HALLMARK_G2M_CHECKPOINT                                                                   3.446443
RETINOBLASTOMA GENE IN CANCER%WIKIPATHWAYS_20181110%WP2446%HOMO SAPIENS                                                     3.389049
RESOLUTION OF SISTER CHROMATID COHESION%REACTOME DATABASE ID RELEASE 66%2500257                                             3.355393
CELL CYCLE, MITOTIC%REACTOME%R-HSA-69278.3                                                                                  3.271655
MITOTIC PROMETAPHASE%REACTOME DATABASE ID RELEASE 66%68877                                                                  3.261985
GTP HYDROLYSIS AND JOINING OF THE 60S RIB

TARGET-30-PANRHJ-01
Series([], Name: NES, dtype: float64)


pathway
REGULATION OF CYTOCHROME-C OXIDASE ACTIVITY%GOBP%GO:1904959             1.682677
POSITIVE REGULATION OF CYTOCHROME-C OXIDASE ACTIVITY%GOBP%GO:1904960    1.682677
POSITIVE REGULATION OF ELECTRON TRANSFER ACTIVITY%GOBP%GO:1904734       1.682677
Name: NES, dtype: float64



TARGET-30-PANZVU-01
pathway
B CELL RECEPTOR SIGNALING PATHWAY%GOBP%GO:0050853                                                                               2.960503
ANTIGEN RECEPTOR-MEDIATED SIGNALING PATHWAY%GOBP%GO:0050851                                                                     2.742906
ANTIGEN ACTIVATES B CELL RECEPTOR (BCR) LEADING TO GENERATION OF SECOND MESSENGERS%REACTOME DATABASE ID RELEASE 66%983695       2.636827
B CELL ACTIVATION%GOBP%GO:0042113                                                                                               2.587924
PID_TCR_PATHWAY%MSIGDB_C2%PID_TCR_PATHWAY                                           

NameError: name 'cluster1_metastatic' is not defined

In [8]:
len(sigs)

2323

In [9]:
for cluster, counts in sub_cohort_unique.items():
    print cluster, len(assign[assign['1'] == cluster])
    
    counter = collections.Counter(counts)
    
    for gs, n in counter.most_common(50):
        print gs, n
    
    print '\n\n'

1 31
T CELL SELECTION%GOBP%GO:0045058 4
PLASMA MEMBRANE INVAGINATION%GOBP%GO:0099024 4
THYMIC T CELL SELECTION%GOBP%GO:0045061 4
BIOCARTA_TCRA_PATHWAY%MSIGDB_C2%BIOCARTA_TCRA_PATHWAY 4
B CELL MEDIATED IMMUNITY%GOBP%GO:0019724 4
POSITIVE REGULATION OF B CELL ACTIVATION%GOBP%GO:0050871 4
PRODUCTION OF MOLECULAR MEDIATOR OF IMMUNE RESPONSE%GOBP%GO:0002440 4
COMPLEMENT ACTIVATION, CLASSICAL PATHWAY%GOBP%GO:0006958 4
PHAGOCYTOSIS, ENGULFMENT%GOBP%GO:0006911 4
BIOCARTA_TCYTOTOXIC_PATHWAY%MSIGDB_C2%BIOCARTA_TCYTOTOXIC_PATHWAY 4
TCR SIGNALING IN NAIVE CD8+ T CELLS%PATHWAY INTERACTION DATABASE NCI-NATURE CURATED DATA%TCR SIGNALING IN NAIVE CD8+ T CELLS 4
COMPLEMENT ACTIVATION%GOBP%GO:0006956 4
GENERATION OF SECOND MESSENGER MOLECULES%REACTOME DATABASE ID RELEASE 66%202433 4
IMMUNOGLOBULIN PRODUCTION%GOBP%GO:0002377 4
HUMORAL IMMUNE RESPONSE MEDIATED BY CIRCULATING IMMUNOGLOBULIN%GOBP%GO:0002455 4
REGULATION OF DENDRITIC CELL DENDRITE ASSEMBLY%GOBP%GO:2000547 4
B CELL PROLIFERATION%GOBP%GO:00421

In [10]:
met = ['POSITIVE REGULATION OF LOCOMOTION%GOBP%GO:0040017',
       'POSITIVE REGULATION OF CELL MIGRATION%GOBP%GO:0030335',
       'POSITIVE REGULATION OF VASCULATURE DEVELOPMENT%GOBP%GO:1904018',
       'REGULATION OF ENDOTHELIAL CELL MIGRATION%GOBP%GO:0010594',
       'POSITIVE REGULATION OF CELLULAR COMPONENT MOVEMENT%GOBP%GO:0051272',
       'POSITIVE REGULATION OF CELL MOTILITY%GOBP%GO:2000147',
       'FOCAL ADHESION%GOCC%GO:0005925']

In [11]:
top5 = []

gs_plot = pd.DataFrame(columns=['cluster', 'gs', 'percent'])
for cluster, counts in sub_clust_unique.items():
    N = len(assign[assign['1'] == cluster])
    
    print cluster, N
    
    counter = collections.Counter(counts)
    
    for gs, n in counter.most_common(100):
        print gs, n
        if cluster == 1:
            top5.append(gs)
        
    print '\n\n'

1 31
FOLDING OF ACTIN BY CCT TRIC%REACTOME DATABASE ID RELEASE 66%390450 2
SYNAPSE ORGANIZATION%GOBP%GO:0050808 1
TCA CYCLE%PANTHER PATHWAY%P00051 1
BIOCARTA_EIF_PATHWAY%MSIGDB_C2%BIOCARTA_EIF_PATHWAY 1
COOPERATION OF PDCL (PHLP1) AND TRIC CCT IN G-PROTEIN BETA FOLDING%REACTOME%R-HSA-6814122.2 1
NCRNA METABOLIC PROCESS%GOBP%GO:0034660 1
REGULATION OF CELLULAR PROTEIN CATABOLIC PROCESS%GOBP%GO:1903362 1
BIOCARTA_CK1_PATHWAY%MSIGDB_C2%BIOCARTA_CK1_PATHWAY 1
SULFUR AMINO ACID METABOLISM%REACTOME DATABASE ID RELEASE 66%1614635 1
MONOCYTE DIFFERENTIATION%GOBP%GO:0030224 1
COPII VESICLE COATING%GOBP%GO:0048208 1
CENTRAL NERVOUS SYSTEM DEVELOPMENT%GOBP%GO:0007417 1
RIBONUCLEOSIDE DIPHOSPHATE METABOLIC PROCESS%GOBP%GO:0009185 1
ESTABLISHMENT OF INTEGRATED PROVIRAL LATENCY%GOBP%GO:0075713 1
GOLGI VESICLE BUDDING%GOBP%GO:0048194 1
POSITIVE REGULATION OF TRANSLATION%GOBP%GO:0045727 1
BRAIN-DERIVED NEUROTROPHIC FACTOR (BDNF) SIGNALING PATHWAY%WIKIPATHWAYS_20181110%WP2380%HOMO SAPIENS 1
RESPIRATORY

In [12]:
top5

['FOLDING OF ACTIN BY CCT TRIC%REACTOME DATABASE ID RELEASE 66%390450',
 'SYNAPSE ORGANIZATION%GOBP%GO:0050808',
 'TCA CYCLE%PANTHER PATHWAY%P00051',
 'BIOCARTA_EIF_PATHWAY%MSIGDB_C2%BIOCARTA_EIF_PATHWAY',
 'COOPERATION OF PDCL (PHLP1) AND TRIC CCT IN G-PROTEIN BETA FOLDING%REACTOME%R-HSA-6814122.2',
 'NCRNA METABOLIC PROCESS%GOBP%GO:0034660',
 'REGULATION OF CELLULAR PROTEIN CATABOLIC PROCESS%GOBP%GO:1903362',
 'BIOCARTA_CK1_PATHWAY%MSIGDB_C2%BIOCARTA_CK1_PATHWAY',
 'SULFUR AMINO ACID METABOLISM%REACTOME DATABASE ID RELEASE 66%1614635',
 'MONOCYTE DIFFERENTIATION%GOBP%GO:0030224',
 'COPII VESICLE COATING%GOBP%GO:0048208',
 'CENTRAL NERVOUS SYSTEM DEVELOPMENT%GOBP%GO:0007417',
 'RIBONUCLEOSIDE DIPHOSPHATE METABOLIC PROCESS%GOBP%GO:0009185',
 'ESTABLISHMENT OF INTEGRATED PROVIRAL LATENCY%GOBP%GO:0075713',
 'GOLGI VESICLE BUDDING%GOBP%GO:0048194',
 'POSITIVE REGULATION OF TRANSLATION%GOBP%GO:0045727',
 'BRAIN-DERIVED NEUROTROPHIC FACTOR (BDNF) SIGNALING PATHWAY%WIKIPATHWAYS_20181110%WP23

In [13]:
def get_event(event):
    if pd.isnull(event):
        print("NULL")
        return np.nan
    
    events = ['Relapse',
              'Death', 
              'Progression',
              'Event',
              'Second Malignant Neoplasm']
    
    if event == 'Censored':
        return 0
    
    elif event in events:
        return 1 
    
    else:
        raise ValueError(event)
        
def get_vital(vital):
    if pd.isnull(vital):
        print("NULL")
        return np.nan
    
    
    if vital == 'Alive':
        return 0
    
    elif vital == 'Dead':
        return 1
    
    else:
        raise ValueError(vital)

pth = '../data/meta/TARGET_NBL_ClinicalData_Discovery_20170525.xlsx'
clinical = pd.read_excel(pth, index_col=0)

surv = pd.DataFrame(index=assign[assign['1'] == 2].index, 
                    columns=['OS', 'vital', 'EFS', 'event', 'immune'])

for sample in assign[assign['1'] == 1].index:
    root = '-'.join(sample.split('-')[:3])
    if root not in clinical.index:
        print "Missing: ", sample
        continue
    
    OS = clinical.loc[root, 'Overall Survival Time in Days'].item()
    vital = get_vital(clinical.loc[root, 'Vital Status'])
    
    EFS = clinical.loc[root, 'Event Free Survival Time in Days'].item()
    event = get_event(clinical.loc[root, 'First Event'])
    
    
    immune = 1 if sample in cluster1_metastatic else 0
    surv.loc[sample, :] = [OS, vital, EFS, event, immune]

NameError: name 'cluster1_metastatic' is not defined

In [ ]:
import uuid

# Can't figure out how to pass this dataframe
survpth = '../data/mycn-na-tme-signal-subtract-cluster1-survival.tsv'
surv.to_csv(survpth, sep='\t')

%%R -i survpth

require(dpylr)
require(survival)
require(survminer)

surv <- read.table(survpth,
                  sep='\t',
                  header=T)

surv <- dplyr::as_tibble(surv)

%%R

fit <- survfit(Surv(OS, vital) ~ immune, data=surv)
ggsurvplot(fit, conf.int=F, pval=T, risk.table=TRUE)

%%R

fit <- survfit(Surv(EFS, event) ~ immune, data=surv)
ggsurvplot(fit, conf.int=F, pval=T, risk.table=TRUE)